# Imports

In [23]:
import pandas as pd
import pyreadstat as spss
import os
import numpy as np
import random 

# Functions and methods

In [ ]:
# ETL STAGE
def train_val_test_split(db,propTrain=.7,propTest=.5):
    # after train size, the remainder is splitted by propTest
    random.seed(42)    
    # Min and Max grade in database
    min_LG=np.min(db['LastGrade'].values)
    max_LG=np.max(db['LastGrade'].values)
    print('Minimum grade in DB '+str(min_LG))
    print('Maximum grade in DB '+str(max_LG))

    # Selecting random positions for the 3 databases: Train, Test, Validation.    
    trainPosition=[]
    testPosition=[]
    validationPosition=[]
    # making sure all databases have all grades
    for g in range(min_LG,max_LG+1):
        rows=db[db['LastGrade']==g].index.tolist()
        trainPosition=trainPosition+random.sample(rows,k=np.ceil(propTrain*len(rows)).astype(int))
        rows=list(set(rows).difference(set(trainPosition)))
        testPosition=testPosition+random.sample(rows,k=np.ceil(propTest*len(rows)).astype(int))
        validationPosition=validationPosition+list(set(rows).difference(set(testPosition)))

    # Generating databases
    dbTrain=db.loc[trainPosition,:].copy()    
    dbValidation=db.loc[validationPosition,:].copy()
    dbTest=db.loc[testPosition,:].copy()
    return dbTrain, dbValidation, dbTest

# Cleaning data

#

In [2]:
# DATA FOR 2014
DataFile='/Users/fipm/OneDrive/Research/Mineduc/BaseDatosPruebas/2014/2014 - 3o primaria/2014 - 3ro Primaria - Versión Final.sav'
df, meta = spss.read_sav(DataFile, encoding='LATIN1', apply_value_formats=True)


In [14]:
db = df.filter(['Cod_Estab','measure_L','measure_M','Cod_Personal'])

In [16]:
# Beginning of student progression and standarized scores
# FOURTH GRADE STUDENTS

listEstu=db['Cod_Personal'].unique()
Path='/Users/fipm/OneDrive/Research/Mineduc/Sire/'

listDB=[
    'FOTO_ESTUDIANTE_2015_UTF8.csv',
    'FOTO_ESTUDIANTE_2016_UTF8.csv',
    'FOTO_ESTUDIANTE_2017_UFT8.csv',
    'FOTO_ESTUDIANTE_2018_UTF8.csv',
    'FOTO_ESTUDIANTE_2019_UTF8.csv',
    'FOTO_ESTUDIANTE_2020_UTF8.csv',

    ]
listGrados=[4,5,6,7,8,9]
for i in range(0,len(listDB)):
    print(listDB[i])
    opGrade=listGrados[i]
    # 4. FOTO_ESTUDIANTE
    studentDB = pd.read_csv(
        Path+listDB[i]
        ,sep='\t',index_col = False, encoding='utf-8'
        ,usecols=['COD_ESTABLECIMIENTO','COD_PERSONAL','COD_ESTUDIANTE','NOM_GRADO']
        )


    # 41. PREPRIMARIA BILINGUE
    # 42. PREPRIMARIA PARVULOS
    # 43. PRIMARIA DE NIÑOS
    # 44. PRIMARIA DE ADULTOS
    # 45. BASICO  
    # 46. DIVERSIFICADO

    selecionarGrado=opGrade
    # school clasification
    Niveles=[43.,43.,43.,43.,43.,43.,43.,45.,45.,45.]
    grados=['','','','TERCERO','CUARTO','QUINTO' ,'SEXTO' ,'PRIMERO BASICO','SEGUNDO BASICO','TERCERO BASICO']
    grado=grados[selecionarGrado]
    nivel=Niveles[selecionarGrado]

    print('Grado : '+grado)
    print('Nivel : '+str(nivel))


    # finding schools of student digeduca DB in SIRE
    eP=studentDB[studentDB['COD_PERSONAL'].isin(listEstu)].copy()
    eP['Nivel']=(eP['COD_ESTABLECIMIENTO']/1)%100
    condicion=(eP['NOM_GRADO']==grado)
    condicion=condicion*(eP['Nivel']==nivel)        
    eP=eP[condicion] # selecting fourth grade students 
    listSireEstu=eP['COD_PERSONAL'].unique()

    # creating dataframe to store results.
    if i==0:
        toDF=dict()
        toDF['COD_PERSONAL']=listEstu
        PSDF=pd.DataFrame(toDF)
        PSDF['LastGrade']=3
    inDB=PSDF['COD_PERSONAL'].isin(listSireEstu)
    name='G'+str(opGrade)
    PSDF[name]=inDB*1
    PSDF['LastGrade']=PSDF['LastGrade']+PSDF[name]
    print('done '+str(opGrade)+' grade')

FOTO_ESTUDIANTE_2015_UTF8.csv
Grado : CUARTO
Nivel : 43.0
done 4 grade
FOTO_ESTUDIANTE_2016_UTF8.csv
Grado : QUINTO
Nivel : 43.0
done 5 grade
FOTO_ESTUDIANTE_2017_UFT8.csv
Grado : SEXTO
Nivel : 43.0
done 6 grade
FOTO_ESTUDIANTE_2018_UTF8.csv
Grado : PRIMERO BASICO
Nivel : 45.0
done 7 grade
FOTO_ESTUDIANTE_2019_UTF8.csv
Grado : SEGUNDO BASICO
Nivel : 45.0
done 8 grade
FOTO_ESTUDIANTE_2020_UTF8.csv
Grado : TERCERO BASICO
Nivel : 45.0
done 9 grade


In [21]:
db = df.filter(['Cod_Estab','measure_L','measure_M','Cod_Personal'])
db=db.merge(PSDF,left_on='Cod_Personal',right_on='COD_PERSONAL',how='left')
db.drop(columns=['COD_PERSONAL'], inplace=True)
db.to_csv('/Users/fipm/OneDrive/GitHubProjects/grade_progression/raw_data/data.csv',
          index=False)

# starts main program

In [22]:
# Loading data
file_path = os.path.join('..','raw_data','data.csv')
db = pd.read_csv(file_path)

In [ ]:
dbTrain, dbValidation, dbTest = train_val_test_split(db,propTrain=.7,propTest=.5)